# Lecture 4  Advanced Regression Models

In [1]:
# import necessary libraries and specify that graphs should be plotted inline
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline 

# warnings reported for function updates, ignore them
import warnings
warnings.filterwarnings('ignore')

In this practice, we implement three advanced regression models: Polynomial Regression, Ridge Regression, and LASSO Regression. 

**Note: For all the model-relevant syntax, you can google the syntax (in bold) and get its manual.**

## 1. Polynomial Regression
### 1.1 Polynomial Regression Basics
In a polynomial regression, the relationship between $y$ and $x$ is modeled as "$k$<sup>th</sup> degree polynomial" in $x$. 

For $k$<sup>th</sup> degree polynomial, the model is shown as:

<center>$y = \beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_k x^k + \varepsilon$</center>

**Polynomial with Multiple Variables**

Note that when we have multiple variables, say, x1 and x2, the polynomials would be all potential combinations of x1 and x2. For example, a model with 2nd degree polynomial for (x1, x2) would be:


<center>$y = \beta_0 + \beta_1 x_1 + \beta_2 x_1^2 + \beta_3 x_2 + \beta_4 x_2^2 + \beta_5 x_1 x_2 + \varepsilon$</center>

It is obvious that when the degree is higher, and when we have more variables, writing out the polynomials would be extremely tedious. (Fortunately, we do not need to generate the polynomials ourselves in Python.)


### Data Loading and Splitting
As multiple variables will be created in polynomial regression, we use a single input variable $x$ for simplicity. We will use the same dataset in the previous lecture, "house.csv". The dependent variable is **'TOTAL_VALUE'**. The independent variable is **'LOT_SQFT'**.

**Practice:** 
- Let dependent variable be **'TOTAL_VALUE'**. Let independent variable be **'LOT_SQFT'**. 
    - Note: Use Series.to_frame() method to convert Series to DataFrame (i.e., 1D to 2D)
- Split the data into 70% training and 30% test set. Set seed (random_state) = 42.
- Check sample size of training and test set. 

In [2]:
# Data Loading
house = pd.read_csv('house.csv')

# Define X and Y Below, print the shape of house_X
house_X = house['LOT_SQFT']
house_y = house['TOTAL_VALUE']
print(house_X.shape)

# Note that house_X is a 1D array, which cannot fit in sklearn models as x.
house_X = house_X.to_frame()

print(house_X.shape)

(5802,)
(5802, 1)


In [3]:
# Data Splitting, check sample
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(house_X, house_y, 
                                                    test_size = 0.3, random_state = 42)

### 1.2 Polynomial Regression with Scikit-Learn

Estimating polynomial regression takes one extra step compared to linear regression. The reason is that generating polynomials is tedious, and we need a specific function to complete this step. Thus, the first step is to generate polynomials, and the second step is to run the linear regression.

Specifically, the two steps are: 

- First, specify the degree of polynomial regression (i.e., speicfy $k$). Generate variables based on the specified degree. This step is done by creating a new polynomial feature object using syntax: 
<br> <center><span style="font-family:Calibri"> **sklearn.preprocessing.PolynomialFeatures(degree)** </span></center>
    - Use .fit_transform(X) method the get the transfered features.
    - You need to specify the degree (i.e., degree = xxx) before training.

- Second, run a linear regression based on the features generated in the first step. This step is done by: 
<br> <center><span style="font-family:Calibri"> **sklearn.linear_model.LinearRegression()** </span></center>
    - Recall that we have learned its features & functions: .fit, .predict, .score, .coef_, .intercept_

**Practice**
- Assume $k=3$. Run polynomial regression. (Hint: first create the polynomial features, then run the linear regression).
- Obtain and report the mse for test set. (Hint: You need to generate polynomial features for test set to do predictions.)
- Obtain and report the coefficient estimates (include and specify intercept).

In [4]:
# S1. Obtain Polynomial Features 
from sklearn.preprocessing import PolynomialFeatures

## S1.1 Define polynomial generation function and set the degree. Change x to x^0, x, x^2, x^3
degree = 3

poly = PolynomialFeatures(degree)


In [5]:
# S1.2 Obtain the variables: Which set?

## Training set

X_train_poly = poly.fit_transform(X_train)

print(X_train_poly.shape)
# X_train_poly
# X^0 => intercept in LR
## Test set => prediction, performance
X_test_poly = poly.fit_transform(X_test)

(4061, 4)


In [6]:
# S2. Run Linear Regression
from sklearn.linear_model import LinearRegression
# S2.1, define the linear regression function (plug in x^0, x, x^2, x^3, then obtain betas)
lr = LinearRegression()
## S2.2, train the model => 4 Xs
lr.fit(X_train_poly , y_train)

LinearRegression()

In [7]:
# S3. Predict and calculate error
# lr.predict(X_test) # wrong dimension
y_test_pred = lr.predict(X_test_poly)
## Calculate mse
MSE = np.mean((y_test - y_test_pred)**2)

In [8]:
# S4. Report estimates
lr.coef_, lr.intercept_

(array([ 0.00000000e+00,  4.78463125e-02, -1.95203304e-06,  2.87628127e-11]),
 169.33378283967755)

### Make Pipelines (Technical Pre-requisite to Obtain Optimal Degree)

We need to define two functions that comes in a specific order to complete the previous task. In practice, it is recommended to use a "pipeline" to automate this process.

To put two or multiple steps (e.g., PolynomialFeatures and LinearRegression) together with a specific sequence, we create a pipeline object using syntax:
<br> <center><span style="font-family:Calibri"> **sklearn.pipeline.make_pipeline()** </span></center>
- The inputs are the models used in the process. The order of input should be the order of your models/objects.
- The pipeline object will replace your original model for estimation. You can imagine that make_pipeline is putting your models in a bucket following a specific order.

Similar to other models, we can still use  .fit, .fit_transform, .score, .predict and so on.

Specific to make_pipeline(), we can use **.named_steps** to obtain the models inside the bucket. If the model is trained, then all the necessary information can be accessed as well (e.g., coefficients, intercept, etc.) To specify which model you want to look into, use ['Python_Defined_Model_Name']

**Practice: Using make_pipeline for simple polynomial regression**

Use make_pipeline to run the polynomial regression with degree = 3. 
- Define the two steps first. Then put them in a pipeline.
- Train the model using the pipeline you created.
- Obtain the mse for test set.
- Compare the process with the previous practice, what are the differences in the progress?

In [9]:
### In below, use make_pipeline for practice
from sklearn.preprocessing  import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# S1. Define models - two models, polynomial transfer first, then linear regression 

pipe_poly = PolynomialFeatures(degree = 3)
pipe_lr = LinearRegression()

# S2. Apply polynomial regression in pipeline
pipe_poly_reg = make_pipeline(pipe_poly, pipe_lr)

pipe_poly_reg.fit(X_train, y_train)

# S3. Predict and get mse

y_test_pred_pipe = pipe_poly_reg.predict(X_test)

MSE_pipe = np.mean((y_test - y_test_pred_pipe)**2)

**Practice: Getting results from make_pipeline**
- Obtain and report the coefficients, and the intercept.

In [10]:
pipe_poly_reg.named_steps ## Dictionary, {}, LABEL_NAME:VALUES, LABEL_NAME - string
 ## Get all models

{'polynomialfeatures': PolynomialFeatures(degree=3),
 'linearregression': LinearRegression()}

In [11]:
pipe_poly_reg.named_steps['linearregression'] # Get the model

LinearRegression()

In [12]:
pipe_poly_reg.named_steps['linearregression'].coef_

array([ 0.00000000e+00,  4.78463125e-02, -1.95203304e-06,  2.87628127e-11])

In [13]:
print(pipe_poly_reg.named_steps['linearregression'].intercept_)
pipe_poly_reg.named_steps['polynomialfeatures'].degree

169.33378283967755


3

### 1.3 Hyperparameter Tuning with Polynomial Regression
In the previous case, we consider a naive scenario where k = 3. Recall that k is the hyperparameter. To get a better prediction result, we may consider tune the degree k to find the k that gives the best performance.

To make best use of our data, and to avoid overfitting, we will apply cross-validation for performance measure. The best k should be chosen based on the (mean) performance of the validation set.


Sklearn provides a nice syntax that combines grid search and cross validation:
<br> <center><span style="font-family:Calibri"> **sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring,  cv)** </span></center>

- estimator : the model. If a sequence of models, then use pipeline to put them together.
- param_grid : dictionary format. Specifies the potential choice of parameters. The keys must be correct.
- scoring : Performance measure. For linear models, default is R-square. Can be specified to mae (neg_mean_absolute_error), mse ('neg_mean_squared_error') as well.
- cv : Determines the cross-validation splitting strategy. If cv is integer (say, k), then k-fold cv. Default is 5-fold cv.


**Practice: Use GridSearchCV with polynomial regression - Train and Predict**

Apply grid search for hyperparameter tuning, and select the best model based on cross-validation performance. Use R-square as the performance measure. Potential candidate of hyperparameter: integers from 1 to 5, include 1 and 5.
- Define grid of hyperparameters.
- Define the estimator.
- Define gridsearchCV
- Train the model. What is the test mse? Is the model chosen based on the test mse?
- By default, GridSearchCV

In [15]:
# load gridsearchCV
from sklearn.model_selection import GridSearchCV

# Load other modules
from sklearn.preprocessing  import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

# S1. Define grid of parameter. 
## The name should be exactly the same, otherwise cannot find which to specify.

param_poly = {'polynomialfeatures__degree' :  range(1,6)  } # Dictionary

## range(i,j): intergers from i to j, not include j
# param_poly = { 'polynomialfeatures__degree'  :  [1, 2, 3, 4, 5] }

# S2. Define estimator: use make_pipeline to combine two functions. 
grid_poly = PolynomialFeatures()
lr_poly = LinearRegression()
pipe_grid_poly = make_pipeline(  grid_poly  ,  lr_poly )

# S3. Define GridSearchCV Estimation function, then train the model
grid_poly_reg = GridSearchCV(pipe_grid_poly, param_poly, cv = 5)

grid_poly_reg.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'polynomialfeatures__degree': range(1, 6)})

In [16]:
# Prediction
y_test_pred_grid = grid_poly_reg.predict(X_test)
np.mean((y_test - y_test_pred_grid)**2)

7045.94615119024

**Outputs of GridSearchCV:**
- best_score_ : the mean validation score of the best model. The best model's performance measure, based on which the model is chosen.
- best_params_: the choice of hyperparameter
- best_estimator_: the best model of choice (and corresponding results). You can access model estimates from this attribute.
- cv_results_: detailed results stored (e.g., time & score of each hyperparameter, each iteration). Dictionary format.

**Practice: Collect Results from GridSearchCV**
- What is the chosen degree of polynomial regression?
- For the best model, report its performance score based on which the model is chosen.
- Explore attribute: cv_results_. Can you provide evidence of why the best model should be chosen? 
- Explore attribute:best_estimator_. Under the chosen model, what are the coefficients (include intercept)?

In [17]:
## 1. Chosen degree: this is the ...? model chosen
grid_poly_reg.best_params_

{'polynomialfeatures__degree': 3}

In [18]:
## 2. The performance is chosen based on ...? mean validation score => for model selection
grid_poly_reg.best_score_

0.3173624288225963

In [19]:
## 3. Check cv_results_
grid_poly_reg.cv_results_

{'mean_fit_time': array([0.00438919, 0.00379572, 0.00259962, 0.0023952 , 0.00299926]),
 'std_fit_time': array([1.01725217e-03, 4.01602055e-04, 4.94529529e-04, 4.90624294e-04,
        1.37734147e-05]),
 'mean_score_time': array([0.00139647, 0.00154204, 0.00119019, 0.00119557, 0.00139666]),
 'std_score_time': array([0.00048848, 0.00070975, 0.00040143, 0.00039953, 0.00049047]),
 'param_polynomialfeatures__degree': masked_array(data=[1, 2, 3, 4, 5],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'polynomialfeatures__degree': 1},
  {'polynomialfeatures__degree': 2},
  {'polynomialfeatures__degree': 3},
  {'polynomialfeatures__degree': 4},
  {'polynomialfeatures__degree': 5}],
 'split0_test_score': array([0.28503762, 0.31455706, 0.32289995, 0.32421681, 0.32007099]),
 'split1_test_score': array([0.26901215, 0.30245627, 0.30871849, 0.3069042 , 0.29903862]),
 'split2_test_score': array([0.30160033, 0.32395589, 0.33977123,

In [40]:
np.max(grid_poly_reg.cv_results_['mean_test_score'])

0.3173624288225963

In [21]:
grid_poly_reg.best_estimator_.named_steps

{'polynomialfeatures': PolynomialFeatures(degree=3),
 'linearregression': LinearRegression()}

In [45]:
## 4. Check best_estimator_ # details of chosen model
grid_poly_reg.best_estimator_.named_steps['polynomialfeatures'].degree
grid_poly_reg.best_estimator_.named_steps['linearregression'].coef_

array([ 0.00000000e+00,  4.78463125e-02, -1.95203304e-06,  2.87628127e-11])

In [46]:
# Unbiased performance measure of best model (in R2)
## test set
grid_poly_reg.score(X_test, y_test)

0.3235362995692007

## Ridge Regression

Estimating ridge regression is done by syntax:
<br> <center><span style="font-family:Calibri"> **sklearn.linear_model.Ridge()** </span></center>
The hyperparameter is specified as "alpha". By default, alpha = 1. *Note that this is the same for almost all models in sklearn.linear_model, including logistic regression*


**Practice**
- Prepare data as in polynomial case. Let $X$ be variables:  'GROSS_AREA', 'ROOMS', 'LIVING_AREA', 'LOT_SQFT', 'FLOORS', 'FULL_BATH'
- Run ridge regression without model selection or CV. Use defalt alpha = 1
- Run ridge regression with grid search and CV. Select tuning parameter from: [0.001, 0.01, 0.1, 1, 10,100].


In [22]:
# Data Loading and Splitting
var = ['GROSS_AREA', 'ROOMS', 'LIVING_AREA', 'LOT_SQFT', 'FLOORS', 'FULL_BATH']
X = house[var]
y = house['TOTAL_VALUE']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)


In [24]:
# Model and prediction
from sklearn.linear_model import Ridge
base_ridge = Ridge(alpha = 1)
base_ridge.fit(X_train, y_train)

base_ridge.coef_, base_ridge.intercept_

(array([3.19378284e-02, 7.32326341e-01, 5.99725568e-02, 8.74173838e-03,
        4.46372940e+01, 1.78431657e+01]),
 41.12660659425967)

In [25]:
base_ridge.predict(X_test)

array([310.25428646, 304.5388758 , 299.95204066, ..., 322.39270423,
       544.48795373, 361.31768735])

In [26]:
base_ridge.score(X_test, y_test)

0.7787011510824884

In [27]:
## Grid Search with CV

# 1. Define a list of parameters (key is 'alpha')
ridge_params = { 'alpha'  : [0.001, 0.01, 0.1, 1, 10,100]  }

# 2. Define function and fit the data
my_ridge = Ridge() # estimator
grid_ridge_cv = GridSearchCV(my_ridge, ridge_params, cv = 5)
grid_ridge_cv.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100]})

In [35]:
# 3.1 Present performance measure
# 3.1.0. Unbiased performance measure
print('Unbiased performance in R2:', grid_ridge_cv.score(X_test, y_test))

# 3.1.1 Mean Validation Score - choose best model
print('Score based on which model is chosen:', grid_ridge_cv.best_score_)

# 3.2 find best hyperparameters
print('Best hyperparameter chosen:', grid_ridge_cv.best_params_)

# 3.3 find best parameter estimates (coefficients)
grid_ridge_cv.best_estimator_.coef_, grid_ridge_cv.best_estimator_.intercept_


Unbiased performance in R2: 0.7787011510824884
Score based on which model is chosen: 0.783214249774319
Best hyperparameter chosen: {'alpha': 1}


(array([3.19378284e-02, 7.32326341e-01, 5.99725568e-02, 8.74173838e-03,
        4.46372940e+01, 1.78431657e+01]),
 41.12660659425967)

## LASSO

Estimating LASSO regression is done by creating a Lasso object using syntax:
<br> <center><span style="font-family:Calibri"> **sklearn.linear_model.Lasso** </span></center>
- hyperparameter is also alpha. Default is 1.

**Practice**
- Prepare data as in polynomial case. Let $X$ be 'GROSS_AREA', 'ROOMS', 'LIVING_AREA', 'LOT_SQFT', 'FLOORS', 'FULL_BATH'.
- Run LASSO regression without model selection or CV
- Run LASSO regression with grid search and CV. Select tuning parameter from: [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10].

In [36]:
# Model and prediction with default hyperparameter
from sklearn.linear_model import Lasso

base_lasso = Lasso(alpha = 1)

base_lasso.fit(X_train, y_train)

base_lasso.coef_, base_lasso.intercept_

(array([2.97487429e-02, 4.80937233e-01, 6.92872333e-02, 8.61525302e-03,
        3.66762713e+01, 1.25785484e+01]),
 54.8229900752886)

In [1]:
# Grid Search with CV - LASSO Case
# 1. Define a list of parameters (key is 'alpha')
lasso_params = { 'alpha'  : [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]  }
# 2. Define function and fit the data
my_lasso = Lasso() # estimator
grid_lasso_cv = GridSearchCV(my_lasso, lasso_params, cv = 5)
grid_lasso_cv.fit(X_train, y_train)

NameError: name 'Lasso' is not defined

In [38]:
# 3.1 Present performance measure
# 3.1.0. Unbiased performance measure
print('Unbiased performance in R2:', grid_lasso_cv.score(X_test, y_test))
# 3.1.1 Mean Validation Score - choose best model
print('Score based on which model is chosen:', grid_lasso_cv.best_score_)
# 3.2 find best hyperparameters
print('Best hyperparameter chosen:', grid_lasso_cv.best_params_)
# 3.3 find best parameter estimates (coefficients)
grid_lasso_cv.best_estimator_.coef_, grid_lasso_cv.best_estimator_.intercept_

Unbiased performance in R2: 0.7787051811426311
Score based on which model is chosen: 0.7832136124199647
Best hyperparameter chosen: {'alpha': 1e-05}


(array([3.19645211e-02, 7.25754110e-01, 5.98952703e-02, 8.74321785e-03,
        4.47262062e+01, 1.78725451e+01]),
 41.02602342600778)